In [ ]:
from glob import glob
import os
from pathlib import Path
from PipeLine import *
from dask.distributed import Client
from dask import delayed
from dask import compute
client = Client(n_workers=4)
client

In [ ]:
# Path to bug extraction folders/files
database = '/Users/amirhsi_mini/trjs_extractions/'
bug_groups = glob(database+"N*_bug/")
geometry = 'cylindrical'

In [ ]:
def analyze_bug(extraction_files, geometry, group_save_to, group_ens_avg_save_to, old_naming=True):
    rhist_name = 'rHists'

    rhist_files = PipeLine.file_reader(extraction_files,extensions=['_rHists.csv','_rEdges.csv'])
    rhist_dicts = PipeLine.ensemble(rhist_files,rhist_name,geometry,dtype=int,single=False,save_to=group_save_to)
    ens_evg_dict_rhist = PipeLine.group(rhist_dicts,rhist_name,geometry,save_to=group_ens_avg_save_to)

    zhist_name = 'zHists'
    zhist_files = PipeLine.file_reader(extraction_files,extensions=['_zHists.csv','_zEdges.csv'])
    zhist_dicts = PipeLine.ensemble(zhist_files,zhist_name,geometry,dtype=int,single=False,save_to=group_save_to)
    ens_evg_dict_zhist = PipeLine.group(zhist_dicts,zhist_name,geometry,save_to=group_ens_avg_save_to)

    thetahist_name = 'thetaHists'
    thetahist_files = PipeLine.file_reader(extraction_files,extensions=['_thetaHists.csv','_thetaEdges.csv'])
    thetahist_dicts = PipeLine.ensemble(thetahist_files,thetahist_name,geometry,dtype=int,single=False,save_to=group_save_to)
    ens_evg_dict_thetahist = PipeLine.group(thetahist_dicts,thetahist_name,geometry,save_to=group_ens_avg_save_to)

    rFloryhist_name = 'rFloryHists'
    rFloryhist_files = PipeLine.file_reader(extraction_files,extensions=['_rFloryHists.csv','_rFloryEdges.csv'])
    rFloryhist_dicts = PipeLine.ensemble(rFloryhist_files,rFloryhist_name,geometry,dtype=int,single=False,save_to=group_save_to)
    ens_evg_dict_rFloryhist = PipeLine.group(rFloryhist_dicts,rFloryhist_name,geometry,save_to=group_ens_avg_save_to)

    fsd_name = 'fsd_t'
    fsd_files = PipeLine.file_reader(extraction_files,extensions=['_fsd_t.csv'])
    fsd_dicts = PipeLine.ensemble(fsd_files,fsd_name,geometry,save_to=group_save_to)
    ens_evg_dict_fsd = PipeLine.group(fsd_dicts,fsd_name,geometry,save_to=group_ens_avg_save_to)

    gyr_name = 'gyr_t'
    gyr_files = PipeLine.file_reader(extraction_files,extensions=['_gyr_t.csv'])
    gyr_dicts = PipeLine.ensemble(gyr_files,gyr_name,geometry,save_to=group_save_to)
    ens_evg_dict_gyr = PipeLine.group(gyr_dicts,gyr_name,geometry,save_to=group_ens_avg_save_to)

    rFlory_name = 'rFlory_t'
    rFlory_files = PipeLine.file_reader(extraction_files,extensions=['_rFlory_t.csv'])
    rFlory_dicts = PipeLine.ensemble(rFlory_files,rFlory_name,geometry,save_to=group_save_to)
    ens_evg_dict_rFlory = PipeLine.group(rFlory_dicts,rFlory_name,geometry,save_to=group_ens_avg_save_to)

    properties_files = PipeLine.file_reader(extraction_files,extensions=['_properties.csv'])
    properties_df = PipeLine.properties(properties_files, geometry,save_to=group_save_to)
    # N2000D15.0ac1.0-extraction_bug: old_naming: True
    # N2000D15.0ac2.0-extraction_bug: old_naming: True
    # N2000D20.0ac1.0-extraction_bug: old_naming: True
    # N2000D20.0ac2.0-extraction_bug: old_naming: True
    # N2000D25.0ac1.0-extraction_bug: old_naming: True
    # N2000D25.0ac2.0-extraction_bug: old_naming: True
    # N2000D25.0ac4.0-extraction_bug: old_naming: True
    # N2000D30.0ac4.0-extraction_bug: old_naming: True
    # N2000D30.0ac6.0-extraction_bug: old_naming: True
    ens_avg_properties = PipeLine.group_properties(properties_df, geometry,old_naming=old_naming,save_to=group_ens_avg_save_to)

    all_rho_r_dicts, all_phi_r_dicts = PipeLine.distributions_generator(rhist_dicts, properties_df, 'dmon', geometry, 'radial', '',normalized=False,save_to=group_save_to)
    ens_avg_dict_rho_r = PipeLine.group(all_rho_r_dicts,'rRhos',geometry,save_to=group_ens_avg_save_to)
    ens_avg_dict_phi_r = PipeLine.group(all_phi_r_dicts,'rPhis',geometry,save_to=group_ens_avg_save_to)

    all_rho_z_dicts, all_phi_z_dicts = PipeLine.distributions_generator(zhist_dicts, properties_df, 'dmon', geometry, 'longitudinal', '',normalized=False,save_to=group_save_to)
    ens_avg_dict_rho_z = PipeLine.group(all_rho_z_dicts,'zRhos',geometry,save_to=group_ens_avg_save_to)
    ens_avg_dict_phi_z = PipeLine.group(all_phi_z_dicts,'zPhis',geometry,save_to=group_ens_avg_save_to)

In [ ]:
def whole_bug_group_analyze(group_files_path, parent_dir, simulation_type, geometry):
    sim_files = glob(group_files_path+"/N*/N*.csv")
    parent_path = group_files_path.split('trjs_extraction')[0]+"/"+parent_dir+"/"
    
    group_name = group_files_path.split("/")[-2].split('-')[0]+'-'+simulation_type+'-analysis'
    group_path = parent_path+group_name
    Path(group_path).mkdir(parents=True, exist_ok=False)
    group_save_to = group_path+"/"
    
    group_ens_avg_name = group_files_path.split("/")[-2].split('-')[0]+'-'+simulation_type+'-analysis-ens_avg'
    group_ens_avg_path = parent_path+group_ens_avg_name
    Path(group_ens_avg_path).mkdir(parents=True, exist_ok=False)
    group_ens_avg_save_to = group_ens_avg_path+"/"
    
    analyze_bug(sim_files,geometry, group_save_to, group_ens_avg_save_to)

In [ ]:
# analyze all the bug groups at once:
simulation_type = 'bug' # bug or all
parent_dir = "trjs_analysis_bug"
geometry = 'cylindrical'
analysis_delayed = []
for bug_group in bug_groups:
    analysis = delayed(whole_bug_group_analyze)(bug_group, parent_dir, simulation_type, geometry)
    analysis_delayed.append(analysis)

In [ ]:
%%time
results = compute(analysis_delayed)

In [ ]:
# analyze log files
log_files = glob('./N*.log')
geometry = 'cylinder'
log_files = PipeLine.file_reader(log_files,extensions=['.log'])
details_out , runtime_out = PipeLine.log_outputs(log_files[0][0], geometry)
PipeLine.lammps_log_details(log_files, details_out , runtime_out)

In [ ]:
# create one dataframe of all the properties files:
database_path = '/Users/amirhsi_mini/trjs_analysis_bug/'
properties_files = glob(database_path+"N*-bug-analysis/N*-properties.csv")
properties_files = PipeLine.file_reader(properties_files,extensions=['-properties.csv'])
properties_all_in_one = PipeLine.all_dfs_in_one(properties_files, 'properties', save_to=database_path, index_col=0)
properties_files_ens_avg = glob(database_path+"N*-bug-analysis-ens_avg/N*-properties-ens_avg.csv")
properties_files_ens_avg = PipeLine.file_reader(properties_files_ens_avg,extensions=['-properties-ens_avg.csv'])
properties_all_in_one_ens_avg = PipeLine.all_dfs_in_one(properties_files_ens_avg,'properties', ens_avg=True, norm_func=PipeLine.cyl_sumrule_norms_ens_evg, save_to=database_path, index_col=0)